# 🚗 Addestramento Record Linkage (Ottimizzato per Esecuzione Locale)

Questo notebook è stato ottimizzato per essere leggero sulla CPU e gestire correttamente i file CSV locali.

### Caratteristiche principali:
- **Gestione Percorsi Hardware**: Rilevamento automatico della cartella `data/`.
- **Efficienza Memoria**: Caricamento selettivo delle colonne e pulizia della cache.
- **Visualizzazione Dati**: Visualizzazione chiara dei file CSV prima dell'esecuzione.

## 1. Setup e Caricamento Dati
Inizializziamo l'ambiente e carichiamo i dati con ottimizzazioni di memoria.

In [4]:
import pandas as pd
import numpy as np
import recordlinkage
import os
import gc
import random
from IPython.display import display

# --- CONFIGURAZIONE PERCORSI ---
# Cerchiamo la cartella data partendo dalla root del progetto
base_path = os.getcwd()
if 'notebooks' in base_path:
    data_dir = os.path.join('..', 'data', 'processed')
    results_dir = os.path.join('..', 'data', 'results')
else:
    data_dir = os.path.join('data', 'processed')
    results_dir = os.path.join('data', 'results')

cl_path = os.path.join(data_dir, 'craigslist_final.csv')
us_path = os.path.join(data_dir, 'us_cars_final.csv')

SAMPLE_SIZE = 50000  # Ridotto per efficienza locale, aumenta se necessario

print(f"🔍 Ricerca dati in: {os.path.abspath(data_dir)}")

cols = ['make', 'model', 'year', 'fuel_type', 'transmission', 'body_type']

try:
    # Caricamento ottimizzato: solo colonne necessarie e tipi di dati efficienti
    df_cl = pd.read_csv(cl_path, index_col='id_cl', usecols=['id_cl'] + cols)
    df_us = pd.read_csv(us_path, index_col='id_us', usecols=['id_us'] + cols)
    
    # Campionamento per ridurre il carico computazionale
    df_cl = df_cl.sample(n=min(SAMPLE_SIZE, len(df_cl)), random_state=42)
    df_us = df_us.sample(n=min(SAMPLE_SIZE, len(df_us)), random_state=42)

    print("✅ Dati caricati con successo!")
    
    # VISUALIZZAZIONE CSV
    print("\n--- Anteprima Craigslist Data ---")
    display(df_cl.head())
    
    print("\n--- Anteprima US Cars Data ---")
    display(df_us.head())
    
    print(f"\nDimensioni Dataset: Craigslist={df_cl.shape}, US Cars={df_us.shape}")

except FileNotFoundError:
    print(f"❌ ERRORE: File non trovati in {data_dir}. Verifica la posizione dei CSV.")
    df_cl, df_us = None, None
except Exception as e:
    print(f"❌ ERRORE IMPREVISTO: {e}")
    df_cl, df_us = None, None

IndentationError: expected an indented block after 'try' statement on line 28 (1241438918.py, line 31)

## 2. Strategie Record Linkage (Manual)
Eseguiamo i test B1 e B2 con gestione della memoria.

In [2]:
import recordlinkage
import pandas as pd
import os
import gc

def run_manual_rl_safe(strategy='B1'):
    if df_cl is None or df_us is None:
        print("❌ Dati non pronti. Carica prima i dataset.")
        return

    print(f"\n🚀 Inizio Record Linkage - Strategia: {strategy}")
    
    # Inizializzazione dell'indexer per il blocking 
    indexer = recordlinkage.Index()
    
    # --- INTEGRAZIONE DELLE STRATEGIE B1 E B2 (Punto 4.D) ---
    if strategy == 'B1':
        # Strategia più ampia: marca e anno [cite: 20]
        indexer.block(['make', 'year']) 
    else:
        # Strategia B2: più restrittiva per massimizzare la precision [cite: 20]
        indexer.block(['make', 'model', 'year']) 
    
    # Generazione dei link candidati
    links = indexer.index(df_cl, df_us)
    print(f"🔹 Candidati individuati: {len(links)}")

    # Definizione delle regole di confronto [cite: 21]
    comp = recordlinkage.Compare()
    comp.string('model', 'model', method='jarowinkler', threshold=0.85, label='model')
    comp.exact('fuel_type', 'fuel_type', label='fuel')
    comp.exact('transmission', 'transmission', label='transmission')

    # --- SOLUZIONE AL MEMORY ERROR: CHUNKING ---
    chunk_size = 1000000 
    all_matches = []

    print(f"⏳ Elaborazione in corso in blocchi da {chunk_size}...")
    for i in range(0, len(links), chunk_size):
        chunk = links[i:i + chunk_size]
        
        # Calcolo delle feature per il blocco attuale
        features_chunk = comp.compute(chunk, df_cl, df_us)
        
        # Calcolo dello score pesato
        features_chunk['score'] = (features_chunk['model'] * 3 + 
                                   features_chunk['fuel'] * 0.5 + 
                                   features_chunk['transmission'] * 0.5)
        
        # Salvataggio solo dei record sopra la soglia di confidenza
        matches_chunk = features_chunk[features_chunk['score'] >= 3.0].reset_index()
        all_matches.append(matches_chunk)
        
        # Pulizia della memoria
        del features_chunk
        gc.collect()

    # Unione dei risultati e salvataggio
    if all_matches:
        final_matches = pd.concat(all_matches, ignore_index=True)
        final_matches.rename(columns={'level_0': 'id_cl', 'level_1': 'id_us'}, inplace=True)
        
        os.makedirs(results_dir, exist_ok=True)
        out_path = os.path.join(results_dir, f'matches_rl_{strategy}.csv')
        final_matches[['id_cl', 'id_us', 'score']].to_csv(out_path, index=False)
        print(f"✅ Completato! Risultati salvati in: {out_path} ({len(final_matches)} record)")
    else:
        print(f"⚠️ Nessun match trovato per la strategia {strategy}.")

# --- ESECUZIONE DELLE DUE PIPELINE (Punto 4.H) ---
run_manual_rl_safe('B1')
run_manual_rl_safe('B2')


🚀 Inizio Record Linkage - Strategia: B1
🔹 Candidati individuati: 9277512
⏳ Elaborazione in corso in blocchi da 1000000...
✅ Completato! Risultati salvati in: ..\data\results\matches_rl_B1.csv (1126263 record)

🚀 Inizio Record Linkage - Strategia: B2
🔹 Candidati individuati: 506212
⏳ Elaborazione in corso in blocchi da 1000000...
✅ Completato! Risultati salvati in: ..\data\results\matches_rl_B2.csv (506212 record)


## 3. Dedupe Training (Machine Learning)
Versione ottimizzata per non saturare la CPU locale.

In [ ]:
import pandas as pd
import dedupe
import csv
import os
import gc
import random

def run_dedupe_with_gt(df_cl, df_us, results_dir, gt_train_path, gt_negatives_path=None):
    if df_cl is None or df_us is None:
        print("❌ Dati non pronti.")
        return

    print(f"DEBUG: Indici df_cl: {len(df_cl)} | df_us: {len(df_us)}")

    os.makedirs(results_dir, exist_ok=True)
    settings_file = os.path.join(results_dir, 'dedupe_learned_settings')
    training_file = os.path.join(results_dir, 'dedupe_training.json')

    fields = [
        dedupe.variables.String('make', has_missing=True),
        dedupe.variables.String('model', has_missing=True),
        dedupe.variables.Exact('year', has_missing=True),
        dedupe.variables.String('body_type', has_missing=True)
    ]

    def to_dedupe_dict(df, fields_list):
        field_names = [f.field for f in fields_list]
        data_dict = {}
        for idx, row in df.iterrows():
            record = {field: (str(row.get(field)) if pd.notna(row.get(field)) else None) for field in field_names}
            clean_id = str(int(idx)) if isinstance(idx, (int, float)) else str(idx)
            data_dict[clean_id] = record
        return data_dict

    print("🔄 Conversione dizionari...")
    data_1 = to_dedupe_dict(df_cl, fields)
    data_2 = to_dedupe_dict(df_us, fields)

    if os.path.exists(settings_file):
        print(f"📂 Caricamento impostazioni esistenti...")
        with open(settings_file, 'rb') as f:
            linker = dedupe.StaticRecordLink(f)
    else:
        linker = dedupe.RecordLink(fields)
        
        labeled_examples = {'match': [], 'distinct': []}
        print(f"📖 Lettura Ground Truth...")
        gt_df = pd.read_csv(gt_train_path)
        
        for _, row in gt_df.iterrows():
            id_1, id_2 = str(int(row['id_cl'])), str(int(row['id_us']))
            if id_1 in data_1 and id_2 in data_2:
                pair = (data_1[id_1], data_2[id_2])
                if int(row.get('label', 1)) == 1:
                    labeled_examples['match'].append(pair)
                else:
                    labeled_examples['distinct'].append(pair)

        # Generazione esempi negativi automatica
        if len(labeled_examples['distinct']) < 500:
            ids_1, ids_2 = list(data_1.keys()), list(data_2.keys())
            while len(labeled_examples['distinct']) < 1000:
                i1, i2 = random.choice(ids_1), random.choice(ids_2)
                if data_1[i1]['make'] != data_2[i2]['make']:
                    labeled_examples['distinct'].append((data_1[i1], data_2[i2]))

        print(f"✅ Coppie pronte: {len(labeled_examples['match'])} Match, {len(labeled_examples['distinct'])} Distinct.")

        # --- MODIFICA CRITICA QUI ---
        print("🧪 Preparazione training...")
        # Usiamo come sample_size il numero totale di record che abbiamo caricato
        # Questo garantisce che Dedupe indicizzi TUTTI i record, inclusi quelli della GT
        total_sample = max(len(data_1), len(data_2))
        linker.prepare_training(data_1, data_2, sample_size=total_sample)
        
        try:
            linker.mark_pairs(labeled_examples)
        except Exception as e:
            print(f"⚠️ Warning: Problema con mark_pairs: {e}")
            print("Provo a resettare l'indice di training...")
            # Fallback estremo: forziamo un numero ancora più alto
            linker.prepare_training(data_1, data_2, sample_size=len(data_1) + len(data_2))
            linker.mark_pairs(labeled_examples)

        print("🧠 Addestramento modello...")
        linker.train()

        with open(training_file, 'w') as tf:
            linker.write_training(tf)
        with open(settings_file, 'wb') as sf:
            linker.write_settings(sf)

    print("🔗 Esecuzione Join...")
    # Il join può essere pesante, usiamo i blocchi (blocking) per non saturare la RAM
    matches = linker.join(data_1, data_2, threshold=0.5)
    
    out_path = os.path.join(results_dir, 'matches_dedupe_final.csv')
    with open(out_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['cl_id', 'us_id', 'confidence'])
        for (id1, id2), score in matches:
            writer.writerow([id1, id2, score])
            
    print(f"✨ Completato! Risultati in: {out_path}")
    gc.collect()

results_dir = "results_integration"
gt_path = os.path.join('..', 'data', 'gt', 'gt_train.csv')

run_dedupe_with_gt(df_cl, df_us, results_dir, gt_path)

DEBUG: Indici df_cl: 50000 | df_us: 50000
🔄 Conversione dizionari...
📖 Lettura Ground Truth...
✅ Coppie pronte: 11 Match, 1000 Distinct.
🧪 Preparazione training...
